## **Lijian Mei**
CS3120 Project

So I using a Steam Game dataset to find what type of features impact sales of games, the type of factors that has the highest influence on if a game sells well. 

Using Kaggle: gamalytic_steam_games.csv https://www.kaggle.com/datasets/safwaibrahim/gamalytic-steam-games-csv/data
and Kaggle: https://www.kaggle.com/datasets/artermiloff/steam-games-dataset?select=games_march2025_cleaned.csv
Merged the two datasets and removed NaN rows.
The file read in Project_merge.csv which is readed into for Analysis.

The goal is to use a RandomForest model to find features that impacted the what type of games sold more copies. The was a lot of factors to take in to account.

In [6]:
import plotly as px
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import glob

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import LabelEncoder  
from sklearn.model_selection import cross_val_score


In [ ]:
#Import data from Merge - created by the two datasets above
df = pd.read_csv('data3/Project_merge.csv', low_memory=False) #From Merge jupter notebook
print('Rows: ', df.shape[0], ' and ', df.shape[1], ' Columns' )

Rows:  84536  and  16  Columns


In [9]:
df.head(3)

,appid,name,release_date,price,developers,publishers,categories,genres,positive,negative,peak_ccu,pct_pos_total,num_reviews_total,copiesSold,publisherClass,reviewScore,developers_encoded,publishers_encoded
0,20,Team Fortress Classic,1999-04-01,4.99,['Valve'],['Valve'],"['Multi-player', 'PvP', 'Online PvP', 'Shared/...",['Action'],7500.0,1121.0,46.0,86.0,6482.0,378635.0,AAA,87.0,46087,40478
1,240,Counter-Strike: Source,2004-11-01,9.99,['Valve'],['Valve'],"['Multi-player', 'Cross-Platform Multiplayer',...",['Action'],172801.0,6697.0,14426.0,96.0,124438.0,15468468.0,AAA,96.0,46087,40478
2,300,Day of Defeat: Source,2010-07-12,9.99,['Valve'],['Valve'],"['Multi-player', 'Cross-Platform Multiplayer',...",['Action'],20604.0,1878.0,285.0,90.0,15155.0,1172320.0,AAA,92.0,46087,40478


Columns - ['developers', 'publishers'] is encode using LabelEncoders for size. 

In [10]:
cols_to_encode = ['developers', 'publishers']
# Dictionary to store LabelEncoders
encoders = {}

for col in cols_to_encode:
    # Initialize encoder
    le = LabelEncoder()
    df[col + '_encoded'] = le.fit_transform(df[col])
    encoders[col] = le

numeric_df = df.select_dtypes(include=['number']).copy()

In [11]:
# Columns ['categories', 'genres', 'publisherClass] are converted one-hot encoded
print('Starting: ', numeric_df.shape)

df['genres'] = df['genres'].str.replace('[', '', regex=False).str.replace(']', '', regex=False)
genre = df['genres'].str.get_dummies(sep=',')
df['categories'] = df['categories'].str.replace('[', '', regex=False).str.replace(']', '', regex=False)
categories = df['categories'].str.get_dummies(sep=',')
publisher = pd.get_dummies(df['publisherClass'], prefix='publisher').astype(int)


numeric_df_final = pd.concat([numeric_df, genre, categories, publisher], axis=1)
print('End: ', numeric_df_final.shape)


Starting:  (84536, 11)
End:  (84536, 120)


In [12]:
print(numeric_df_final.head(3))

   appid  price  positive  negative  peak_ccu  pct_pos_total  \
0     20   4.99    7500.0    1121.0      46.0           86.0   
1    240   9.99  172801.0    6697.0   14426.0           96.0   
2    300   9.99   20604.0    1878.0     285.0           90.0   

   num_reviews_total  copiesSold  reviewScore  developers_encoded  ...  \
0             6482.0    378635.0         87.0               46087  ...   
1           124438.0  15468468.0         96.0               46087  ...   
2            15155.0   1172320.0         92.0               46087  ...   

   'Stats'  'Steam Achievements'  'Steam Cloud'  'Steam Trading Cards'  \
0        0                     0              0                      0   
1        0                     0              0                      0   
2        0                     0              0                      0   

   'Tracked Controller Support'  'VR Only'  publisher_AA  publisher_AAA  \
0                             0          0             0              1   

![Alt text](image/1.PNG "Corr Graph")

Using EDA these are the basic Corr

### My model was RandomForest Regression to find CopiesSold. 
Split my data to test size 0.2

![Alt text](image/2-1.PNG "model_1.1")

![Alt text](image/2.PNG "model_1")

![Alt text](image/2-3.PNG "model_graph1")

Number of reviews had a large impact but not as large as the basic correlation graph.
So what if we removed the num_reviews


# rf2 = RandomForestRegressor(n_estimators=200, max_depth=200)

![Alt text](image/3-1.PNG "model_2")

![Alt text](image/3-2.PNG "model_2_graph")